# Autonomous Driving Software Engineering - Lecture 10: Teleoperation  


## Our objective is to send data over the network
- Understand essential principles of MQTT (Message Queuing Telemetry Transport)
- Why MQTT
- Send our custom message to multiple subscribers

## MQTT  
"MQTT is an OASIS standard. The specification is managed by the OASIS MQTT Technical Committee." - [Mqtt Link](https://mqtt.org/mqtt-specification/)  

![](img/pub_sub_architecture.png)

Picture taken from [HiveMQ GmbH](https://www.hivemq.com/blog/mqtt-essentials-part2-publish-subscribe/) on the 21/04/2021

## Principles of MQTT

- "One" broker and many clients
- Clients publish and/or subscribe to topics
- The broker dispatches messages between sender and the respective receivers

## Why MQTT  

- Efficient and lightweight (usage on microcontrollers + small headers for optimized network bandwidth usage)  
- Bi-directional communications (publish/subscribe instead of client-server)  
- Scalability  
- Reliable message delivery (QoS: 0 = at most once, 1 = at least once, 2 = exactly once)

## A single publish

Sends data to a specific topic

In [1]:
import paho.mqtt.publish as publish
import time

number_of_cycles = 10
sleep_interval_in_sec = 1

for iteration in range(number_of_cycles):
    publish.single("testtopic/single", "payload_" + str(iteration+1),\
                   hostname="broker.emqx.io")
    print("published number: " + str(iteration+1))
    time.sleep(sleep_interval_in_sec)

print("finished publishing")

published number: 1
published number: 2
published number: 3
published number: 4
published number: 5
published number: 6
published number: 7
published number: 8
published number: 9
published number: 10
finished publishing


## Debugging of MQTT topics with web interface

Online tools make it easier to interact with topics. Details about the folllwing steps can be found at [emqx_blog](https://www.emqx.io/blog/mqtt-broker-server). Example usage:

### Create MQTT client via web interface
1. Visit the web interface [emqx_interface](http://tools.emqx.io)
2. Click on "+ New Connection"
3. Type in a Name (e. g. Test)
4. Click on Connect

### Let the web client subscribe to your publisher.py topic
1. Click on new subscription
2. Type as Topic: <em>topic_name</em>
3. Click on Confirm

## A single subscribe

Receives data from specific topic

In [ ]:
import paho.mqtt.subscribe as subscribe

while True:
    msg = subscribe.simple("testtopic/single", hostname="broker.emqx.io")
    print(str(msg.payload.decode("utf-8")))


payload_1
payload_2
payload_3
payload_4
payload_5
payload_6
payload_7
payload_8
payload_9
payload_10


Without ".decode()" function, the content of the bytes object would be printed (b'payload')

## A publisher of our custom class


Let's send a custom message over the network.

In [ ]:
import vehicleParams as veh
q7 = veh.VehicleParams(5.09, 1.98)

<span style="color: red;">**The next cell fails.**</span>  
Only string, bytearray, int, float or None can be sent over network.  


In [ ]:
import paho.mqtt.publish as publish
publish.single("testtopic/single", q7, hostname="broker.emqx.io")

## Serialization
Data is converted into a bytestream

In [ ]:
import pickle
serialized_q7 = pickle.dumps(q7)
#print(serialized_q7)

## Deserialization
Data is converted back from bytestream to original format

In [ ]:
previous_q7 = pickle.loads(serialized_q7)
print("Q7 dimensions: width {} and length {}".\
      format(previous_q7.length, previous_q7.width))

## Serialized publisher
The next program continuously sends serialized data to the specified topic:

In [ ]:
import time
while True:
    publish.single("testtopic/single", serialized_q7, hostname="broker.emqx.io")
    time.sleep(2)

## Deserializing subscriber
Subscriber converts data into readable format

In [ ]:
import vehicleParams
import paho.mqtt.subscribe as subscribe
import pickle

while True:
    msg = subscribe.simple("testtopic/single", hostname="broker.emqx.io")
    unserialized_q7 = pickle.loads(msg.payload)
    print("vehicle length %s and width %s" % \
          (unserialized_q7.length, unserialized_q7.width))

# The client is the main mqtt object
Example taken from [paho-mqtt](https://pypi.org/project/paho-mqtt/#client).  
Further documentation can be found at [eclipse-paho-python](https://www.eclipse.org/paho/index.php?page=clients/python/docs/index.php#connect-reconnect-disconnect)

In [ ]:
import paho.mqtt.client as mqtt

this_program_should_be_terminated = False

def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("testtopic/single")

def on_message(client, userdata, msg):
    payload_text = msg.payload.decode("utf-8")
    print(msg.topic+" "+str(payload_text))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("broker.emqx.io", port=1883, keepalive=10)

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.

client.loop_forever()

## Outlook

Threaded mqtt interface. Main thread is independent of mqtt subscription

In [ ]:
import paho.mqtt.client as mqtt

this_program_should_be_terminated = False

def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("testtopic/single")

def on_message(client, userdata, msg):
    payload_text = msg.payload.decode("utf-8")
    print(msg.topic+" "+str(payload_text))
    finish_statement = "finish"
    if payload_text == finish_statement:
        global this_program_should_be_terminated
        this_program_should_be_terminated = True

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("broker.emqx.io", port=1883, keepalive=10)

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.

client.loop_start()

import time
while True:
    time.sleep(5)
    print("Here, the main work is done")
    if this_program_should_be_terminated:
        print("program terminating")
        break

print("program terminated")